# 8. Análise

Relembrando o objetivo da análise (item 2), queremos construir um panorama do preço por metro quadrado na cidade de São Paulo.

As principais perguntas que queremos responder são:
- Quais são os top 10 bairros de maior preço por metro quadrado na cidade de São Paulo?
- Como tem sido a variação desses valores nos últimos 3 anos? Que bairros apresentam maior valorização?
- Que características dos imóveis (tipologia, número de quartos, vaga de garagem, com/sem elevador, etc) estão relacionadas a maior valorização dos mesmos?
- Existe sazonalidade no mercado imobiliário desta cidade? Quais são os meses mais indicados para compra descontada?
- Como é a aceitação dos vendedores em relação a financiamento? Em média, qual percentual do valor de venda é financiado?

Para responder as perguntas acima, iremos considerar o seguinte recorte:
- Transações de compra e venda
- Imóveis de uso residencial (individual, não prédios)
- Janela de tempo: jan/22 a dez/25


## 8.1 Evolução do preço do metro quadrado da cidade de são Paulo:

In [0]:
%sql
/* Preço do metro quadrado vendido em São Paulo mês a mês, de 2022 a 2025 */

SELECT 
  mes_transacao,
  ROUND(SUM(avg_preco_m2 * num_transacoes) / SUM(num_transacoes), 2) AS avg_preco_m2,
  SUM(num_transacoes) AS num_transacoes

FROM mvp_engdados_puc.ouro.guias_itbi_ouro 

GROUP BY mes_transacao
ORDER BY mes_transacao


Gráfico da evolução do preço médio por metro quadrado vendido na cidade de São Paulo, mês a mês, utilizando dashboard do Databricks:

![Captura de Tela 2025-12-10 às 21.07.03.png](./Captura de Tela 2025-12-10 às 21.07.03.png "Captura de Tela 2025-12-10 às 21.07.03.png")

Gráfico da evolução do preço médio por metro quadrado vendido da cidade de São Paulo, mês a mês, no Google Sheets, com linha de tendência:

![Captura de Tela 2025-12-10 às 21.08.06.png](./Captura de Tela 2025-12-10 às 21.08.06.png "Captura de Tela 2025-12-10 às 21.08.06.png")

O preço médio do metro quadrado vendido na cidade de São Paulo em 2025 (até outubro) foi de R$ 5.545,45. 

De jan/22 a out/25, o aumento foi de R$ 932,51, representando 20,2% de aumento no período.

Podemos notar ainda pelos gráficos que não existe uma sazonalidade bem definida, ao longo dos meses do ano. A tendência é sempre de aumento de preço mês a mês, como podemos notar pela linha de tendência marcada no segundo gráfico (R2 = 0,922).

Em outras palavras, se o objetivo é comprar com desconto, o melhor momento é o mais rápido possível!

## 8.2 Preço do metro quadrado por bairro, 2025:


Para estudar o comportamento do preço de venda de imóveis na granularidade de bairro, precisamos ter cuidado com a amostra selecionada.

Alguns bairros apresentam pequeno número de transações, o que implica em valores muito voláteis (um outlier pode alterar significativamente a média) e portanto pouco confiáveis.

Para evitar esta questão, vamos considerar no estudo apenas bairros com no mínimo 30 transações efetuadas em 2025. De acordo com o Teoremo do Limite Central, esta é a menor amostra para que os valores das transações se aproximem de uma distribuicão normal. 

Na prática do problema que estamos empenhados em resolver, a exclusão deste bairros não causará impacto, uma vez que temos interesse de negócio apenas em bairros de alta liquedez.

Dado o recorte, discutido acima, temos:

In [0]:
%sql
/* Preço do metro quadrado por bairro, em 2025 */
/* Considerando recorte de bairros com pelo menos 30 transações em 2025 */

SELECT 
  bairro,
  ROUND(SUM(avg_preco_m2 * num_transacoes) / SUM(num_transacoes), 2) AS avg_preco_m2,
  SUM(num_transacoes) AS num_transacoes

FROM mvp_engdados_puc.ouro.guias_itbi_ouro 

WHERE mes_transacao BETWEEN '2025-01-01' AND '2025-12-31'

GROUP BY bairro
HAVING SUM(num_transacoes) >=30
ORDER BY avg_preco_m2 DESC

LIMIT 50

Top 10 bairros com maior preço por metro quadrado vendido na cidade de São Paulo, em 2025 (recorte de bairros com pelo menos 30 transações):

![Captura de Tela 2025-12-10 às 21.06.44.png](./Captura de Tela 2025-12-10 às 21.06.44.png "Captura de Tela 2025-12-10 às 21.06.44.png")

Considerando o recorte de bairros com pelo menos 30 transações no período, os top 10 bairros mais caros em preço de venda por metro quadrado na cidade de São Paulo em 2025 são: 
1. Jardim América 
2. Jardim Paulistano 
3. Jardim Europa 
4. Cidade Jardim
5. Itaim Bibi 
6. Vila Nova Conceição 
7. Alto de Pinheiros 
8. Cerquira César 
9. Moema 
10. Vila Olímpia 

## 8.3 Evolução do preço por metro quadrado dos top 10 bairros:

In [0]:
%sql
/* Evolução do preço do metro quadrado dos top 10 bairros, de 2022 a 2025 */
/* Recorte de bairros com pelo menos 30 transações em 2025 */

SELECT 
  YEAR(mes_transacao) AS ano,
  bairro,
  ROUND(SUM(avg_preco_m2 * num_transacoes) / SUM(num_transacoes), 2) AS avg_preco_m2,
  SUM(num_transacoes) AS num_transacoes

FROM mvp_engdados_puc.ouro.guias_itbi_ouro 

WHERE bairro IN 
(
  SELECT bairro 
  FROM
    (
    SELECT 
      bairro,
      ROUND(SUM(avg_preco_m2 * num_transacoes) / SUM(num_transacoes), 2) AS avg_preco_m2,
      SUM(num_transacoes) AS num_transacoes

    FROM mvp_engdados_puc.ouro.guias_itbi_ouro 
    WHERE mes_transacao BETWEEN '2025-01-01' AND '2025-12-31'

    GROUP BY bairro
    HAVING SUM(num_transacoes) >=30
    ORDER BY avg_preco_m2 DESC

    LIMIT 10
    )
) 

GROUP BY ano, bairro
ORDER BY bairro, ano

Gráfico da evolução, ano a ano, do preço por metro quadrado vendido nos top 10 bairros da cidade de São Paulo (definidos no item anterior):

![Captura de Tela 2025-12-10 às 21.29.35.png](./Captura de Tela 2025-12-10 às 21.29.35.png "Captura de Tela 2025-12-10 às 21.29.35.png")

![Captura de Tela 2025-12-10 às 21.31.16.png](./Captura de Tela 2025-12-10 às 21.31.16.png "Captura de Tela 2025-12-10 às 21.31.16.png")

O preço do metro quadrado aumentou em todos os bairros em estudo, no período de 2022 a 2025. Esse aumento variou de 5,7% a 97,6%, sendo a média 21,3%.

O bairro com a maior valorizacão foi o bairro Cidade Jardim.


## 8.4 Panorama da utilização de financiamento imobiliário:

In [0]:
%sql
/* Panorama da utilização de financiamento imobiliário */

SELECT 
  YEAR(a.Data_Transacao) as ano,
  CASE 
    WHEN Tipo_Financiamento = 'nan' THEN "Não"
    ELSE "Sim"
  END AS financiamento,  
  COUNT(*) AS num_transacoes,
  ROUND(SUM(a.Valor_Transacao), 2) AS valor_transacionado,
  ROUND(SUM(a.Valor_Financiado), 2) AS valor_financiado,
  ROUND(SUM(a.Valor_Financiado)/SUM(a.Valor_Transacao), 2) AS percentual_financiado
  
FROM mvp_engdados_puc.prata.guias_itbi_prata AS a

LEFT JOIN mvp_engdados_puc.prata.cep_sp_capital AS b
ON a.CEP = b.CEP

WHERE 1=1 
AND Proporcao_Transmitida = 100 /* Considera apenas imóveis vendidos por inteiro */
AND Uso_IPTU IN /* Filtra apenas imóveis de uso residencial */
  (
  SELECT Uso 
  FROM mvp_engdados_puc.prata.tabela_dim_usos 
  WHERE descricao IN ('RESIDÊNCIA', 'RESIDÊNCIA COLETIVA, EXCLUSIVE CORTIÇO (MAIS DE UMA RESIDÊNCIA NO LOTE)', 'APARTAMENTO EM CONDOMÍNIO (EXIGE FRAÇÃO IDEAL)', 'FLAT RESIDENCIAL EM CONDOMÍNIO (EXIGE FRAÇÃO IDEAL)')
  )

GROUP BY ano, financiamento
ORDER BY ano, financiamento

Gráficos de participação do financiamento nas transacões imobiliárias e percentual do valor do imóvel financiado:

![Captura de Tela 2025-12-10 às 20.27.39.png](./Captura de Tela 2025-12-10 às 20.27.39.png "Captura de Tela 2025-12-10 às 20.27.39.png")

![Captura de Tela 2025-12-10 às 20.41.49.png](./Captura de Tela 2025-12-10 às 20.41.49.png "Captura de Tela 2025-12-10 às 20.41.49.png")

Apenas 36,5% das transações imobiliárias da cidade de São Paulo foram feita através de financiamento imobiliário, desde 2022. Em 2025, essa proporção caiu para 33,5%. 

Dos imóveis que foram financiados, em média 63% do seu valor teve origem no financiamento. Em 2025, esse percentual caiu para 61%.

Como a maior parte dos imóveis são comercializados sem a utilização de financimento imobiliário, e essa proporção caiu no último ano, a apresentação de propostas a vista tem a tendência de ser cada vez mais relevante para competitividade das mesmas. 

## 8.5 Conclusão da análise:


Ao final, deve haver uma discussão de uma forma geral sobre a solução do problema a partir das discussões de cada resposta.

# 9. Autoavaliação

Critérios de avaliação
Objetivo (1,0 pt). O objetivo do trabalho deve ser muito bem detalhado; é um planejamento do trabalho, contendo de forma clara e objetiva o problema a ser resolvido e as perguntas de negócio a serem respondidas. Será avaliada a qualidade desta descrição.
Coleta (0,5 pt). Será avaliada a documentação sobre a coleta dos conjuntos de dados e a persistência dos mesmos na plataforma de nuvem.
Modelagem (2,0 pt). Será avaliada a qualidade da modelagem dos dados (1,0 pt) e documentação do Catálogo de Dados (1,0 pt).
Carga (1,0 pt). Será avaliada a qualidade da documentação da carga dos dados, bem como a corretude e persistência dos dados na plataforma de nuvem após a carga.
Análise (3,0 pt). Serão avaliados a análise de qualidade dos dados (1,0 pt) e da solução do problema de forma correta (0 pt) e bem analisada pela discussão a partir das respostas obtidas (1,0 pt).
Autoavaliação (0,5 pt). Será avaliada a autoavaliação do aluno com as questões pertinentes sobre o atingimento de seus objetivos traçados no início do trabalho.
Capricho (2,0 pt). Aqui serão avaliados o capricho e a qualidade geral do trabalho como um todo de forma subjetiva.